In [1]:
import redcap
import csv
import datetime
import pandas as pd

# REDCap API endpoint and token
data = {
    'token': 'DE12CA04415CF93BCD68321EBF5040FD',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])


In [2]:
df_socio=df.export_records(forms=['socioeconomic_demographic'])
df_lmp=df.export_records(forms=['lmp'])
df_ultr=df.export_records(forms=['ultrasound_scan'])

In [3]:
df_rec=df.export_records(forms=['recruitment'])
df_rec=pd.DataFrame(df_rec)
df_rec=df_rec[['participant_id','rage']]
df_rec=df_rec[
    (df_rec['rage']!='')
].drop_duplicates('participant_id')

In [4]:
df_socio=pd.DataFrame(df_socio)
df_lmp=pd.DataFrame(df_lmp)
df_ultr=pd.DataFrame(df_ultr)

In [5]:
df_socio=df_socio[
    (df_socio['socioeconomic_demographic_complete']=='2')&
    (df_socio['date_form_completed_q47']<='2023-12-1')
]
#df_socio = df_socio.sort_values(by=['participant_id', 'date_form_completed_q47'])
#df_socio= df_socio.drop_duplicates(subset='participant_id', keep='last')


In [6]:

df_lmp=df_lmp[
    (df_lmp['lmp_complete']=='2')&
    (df_lmp['lmp_form_date_q5']<='2023-12-1')
]

df_lmp = df_lmp.sort_values(by=['participant_id', 'lmp_form_date_q5'])
df_lmp= df_lmp.drop_duplicates(subset='participant_id', keep='last')

df_ultr=df_ultr[
    (df_ultr['ultrasound_scan_complete']=='2')&
    (df_ultr['uss_week']=='1')&
    (df_ultr['ultrascan_date']<='2023-12-1')
].drop_duplicates('participant_id')
df_ultr= df_ultr.sort_values(by=['participant_id', 'ultrascan_date'])
df_ultr= df_ultr.drop_duplicates(subset='participant_id', keep='last')


In [7]:
lmp_mergh=pd.merge(df_lmp,df_ultr,on='participant_id',how='right').drop_duplicates('participant_id')
lmp_mergh2=pd.merge(df_socio,lmp_mergh,on='participant_id',how='inner')
lmp_mergh3=pd.merge(df_rec,lmp_mergh2,on='participant_id',how='inner')
lmp_mergh3.drop_duplicates('participant_id')

,participant_id,rage,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,dethinicity_614faf,spe_ethinicity,what_is_your_marital_statu,how_many_co_wives_do_you_h,d_family_size_amt_31c29c,...,abdomen_genitalia,limbs,thorax_2,is_the_morphology_scan_nor,comments_scan,uss_completedby_q33,uss_form_date_q34,upload_uss_report_q35,uss_report_verified_by_q36,ultrasound_scan_complete
0,EDN0009-C,20.33,recruitment_arm_1,,,1,,1,1,2,...,1,1,1,1,Breech presentation with fetal heart rate of ...,27,2023-06-21,20230621_103814.jpg,8,2
1,EDN0017-E,29.97,recruitment_arm_1,,,1,,1,0,6,...,1,1,1,1,,,2022-07-05,,2,2
2,EDN0018-C,21.7,recruitment_arm_1,,,1,,1,1,4,...,1,1,1,1,Normal scan of single life baby.The gestation ...,18,2022-05-31,,2,2
3,EDN0021-A,23.69,recruitment_arm_1,,,6,Konyajo,1,0,4,...,,,,,Pregnancy is still young to assess the morphol...,27,2023-09-19,20230919_101614.jpg,8,2
4,EDN0023-C,19.18,recruitment_arm_1,,,1,,1,0,4,...,,,,1,,28,2022-12-12,,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,EDN1072-V,28.99,recruitment_arm_1,,,1,,1,1,4,...,,,,,CRL is measured and the morphology cannot be a...,27,2023-05-10,20230510_132445.jpg,8,2
245,EDN1079-D,31.39,recruitment_arm_1,,,1,,1,0,15,...,,,,,,18,2023-07-12,20230711_104741.jpg,8,2
246,EDN1080-E,34.53,recruitment_arm_1,,,1,,1,0,20,...,,,,0,,27,2023-03-28,,8,2
247,EDN1104-D,25.64,recruitment_arm_1,,,1,,1,0,6,...,,,,0,,18,2023-10-18,20231018_095615.jpg,,2


In [8]:
lmp_mergh3.to_csv('socio_lmp_ultra_data.csv',index=False)
df_lmp.to_csv('df_lmp.csv',index=False)
df_ultr.to_csv('df_ultr.csv',index=False)